In [47]:
import pandas as pd
import numpy as np

In [48]:
# Para descargar los datos
df = pd.read_csv("vid18_180219.dat",delimiter=',')

In [49]:
# Selecciono las columnas de interés
df = df.filter(['TIMESTAMP','AirTC_Avg'],axis=1)
df.head()

,TIMESTAMP,AirTC_Avg
0,2018-05-11 20:10:00,23.50
1,2018-05-11 20:20:00,22.96
2,2018-05-11 20:30:00,22.73
3,2018-05-11 20:40:00,22.40
4,2018-05-11 20:50:00,22.46


In [50]:
# Aquí hago que TIMESTAMP se presente en formato de fecha
# Además, creo las columnas de año, mes, día y hora

df['TIMESTAMP'] = pd.to_datetime(df.apply(lambda x: x['TIMESTAMP'],1), dayfirst=True)
df['Año']=df['TIMESTAMP'].dt.year
df['Mes']=df['TIMESTAMP'].dt.month
df['Dia']=df['TIMESTAMP'].dt.day
df['Hora']=df['TIMESTAMP'].dt.hour 

In [51]:
df['TempProm'] = df['AirTC_Avg']

In [52]:
# Para tomar los datos desde el primero de Noviembre del 2018
df = df[(df['TIMESTAMP'] >= "2018-11-1")]
df= df.reset_index(drop=True)
df.head()

,TIMESTAMP,AirTC_Avg,Año,Mes,Dia,Hora,TempProm
0,2018-11-01 00:00:00,9.13,2018,11,1,0,9.13
1,2018-11-01 00:10:00,8.89,2018,11,1,0,8.89
2,2018-11-01 00:20:00,8.66,2018,11,1,0,8.66
3,2018-11-01 00:30:00,8.52,2018,11,1,0,8.52
4,2018-11-01 00:40:00,8.47,2018,11,1,0,8.47


In [53]:
# Aquí agrego una columna duplicada de AirTC_Avg y la elimino junto con TIMESTAMP
df['TempProm'] = df['AirTC_Avg']
df=df.drop(['TIMESTAMP','AirTC_Avg'],1)
df.head()

,Año,Mes,Dia,Hora,TempProm
0,2018,11,1,0,9.13
1,2018,11,1,0,8.89
2,2018,11,1,0,8.66
3,2018,11,1,0,8.52
4,2018,11,1,0,8.47


In [54]:
df.dtypes

Año           int64
Mes           int64
Dia           int64
Hora          int64
TempProm    float64
dtype: object